# Generate popularity ranking

# Generate centrality ranking

# Generate prompts for instances

In [ ]:
def generatePromptForInstances():
    """
    This function generates all the prompts needed for testing 
    For these techniques: 0Shot, FewShot, COT

    We need to decide on which properties are we using..
    For now ill be using manual prompts
    this is the user input..
    """
    pass 

# Run models x prompt

In [ ]:
import os 
from google import genai
from google.genai import types
from openai import OpenAI

def runGemini(model,prompt,inputtest):
    

   # ADD API KEY
    client = genai.Client(api_key=API_KEY)

    response = client.models.generate_content(
        model=model,
        config=types.GenerateContentConfig(
            system_instruction=prompt),
            contents=inputtest
    )

    return response.text

def runGpt(model,prompt,inputtest):
    # ADD API_KEY HERE
    from openai import OpenAI
    client = OpenAI(
    )

    response = client.responses.create(
        model=model,
        instructions=prompt,
        input=inputtest,
        store=True,
    )
    return response.output_text

# Sempre manter o mesmo prompt, o que muda é o user input -- que vao ser incorporados as entities e as propriedades pedidas..
# Mas como por nequanto vou manter as mesmas propriedades, 

# TODO: modificar os prompts para pedir que ele siga o que o usuario indicar

def getPrompts(promptsPath="../prompts/"):
    promptsFiles = os.listdir(promptsPath)
    prompts = []

    for filename in promptsFiles:
        if filename.endswith(".txt"):
            file_path = os.path.join(promptsPath, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
                prompts.append(content)

    promptsFiles = [x.replace('.txt','') for x in promptsFiles]
    return promptsFiles, prompts


def getInputs(inputs_path="../inputs/"):
        
    inputFilesTest = os.listdir(inputs_path)
    inputTexts = []

    for filenameInput in inputFilesTest:
        if filenameInput.endswith(".txt"):
            file_path = os.path.join(inputs_path, filenameInput)
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()
                inputTexts.append(content)

    return inputFilesTest, inputTexts



def generateUserInput(props='iata, name, city'):
    """
    Needs to generate all of the possible user inputs (combination of attributes and inputs)
    for now, attributes will be the same.. replace [PROPERTIES] and [INPUT]
    """
    
    userBasePrompt = """"
    These are the attributes extracted from the database for you to enrich the properties:

    [INPUT]
    """

    inputFiles, inputTexts = getInputs()

    completedUserInput = []

    for index, instance in enumerate(inputTexts):
        completedUserPrompt = userBasePrompt.replace('[PROPERTIES]', props).replace('[INPUT]', instance)
        completedUserInput.append(completedUserPrompt)

    inputFiles = [x.replace('.txt','') for x in inputFiles]
    return inputFiles,completedUserInput

MODELS = ['gemini-2.5-pro']
promptsFiles,PROMPTS = getPrompts()
inputsFiles,inputUser = generateUserInput()

PROMPTS = PROMPTS[:3]



# inicialmente vai salvar cada run e cada coisa num diretorio, depois tem um codigo que le todo o diretorio e monta o majority vote e depois 
# monta as métricas
def runTests(n):
    for indexInput, inputtest in enumerate(inputUser):
        for model in MODELS:
            for indexPrompt, prompt in enumerate(PROMPTS):
                for run in range(n):
                    # N runs for a specific prompt of a specific model for the same input
                    if 'gemini' in model:
                        response = runGemini(model, prompt,inputtest)
                    elif 'gpt' in model:
                        response = runGpt(model, prompt, inputtest)
                    output_path = "results_run/"+f"{model}_{inputsFiles[indexInput]}_{promptsFiles[indexPrompt]}_{run}.csv"
                    with open(output_path, "w", encoding="utf-8") as outputFile:
                        outputFile.write(response)

runTests(3)

## Majority vote

In [123]:
# The idea here is the output being model, prompt, results..
import os
import csv
from collections import defaultdict, Counter
import pandas as pd

path = '/results_run'

def constructingDataFrame(results):
    all_keys = set()
    for info in results.values():
        all_keys.update(info.keys())

    columns = ["model"] + sorted(all_keys)
    rows = []
    for model, info in results.items():
        row = {"model": model}
        row.update(info)
        rows.append(row)

    df = pd.DataFrame(rows, columns=columns)
    df.to_csv("airports_dynamic.csv", index=False, encoding="utf-8")

    return df


def calculateMajorityVote(results):
    # Calculates the majority vote for each group
    final_results = {}

    for test_name, prop_map in results.items():
        result = {}

        for prop, values in prop_map.items():
            if len(values) == 0:
                continue
            most_common, count = Counter(values).most_common(1)[0]
            result[prop] = most_common

        final_results[test_name] = result

    return final_results 

def organizeVotes(directory, runs=3):
    """
    Lê todos os arquivos CSV em um diretório.
    Agrupa por prefixo (ex.: gemini_prompt1).
    Faz majority vote dos valores para cada propriedade.
    """

    grouped_votes = defaultdict(lambda: defaultdict(list))

    for filename in os.listdir(directory):
        test = filename[:-5]
        file_path = os.path.join(directory, filename)

        with open(file_path, newline="", encoding="utf-8") as csvfile:
            reader = csv.DictReader(csvfile)
            
            for row in reader:
                prop = row.get("property")
                val = row.get("value")

                if prop is None or val is None:
                    continue

                prop = prop.strip()
                val = val.strip()
                if prop and val:
                    grouped_votes[test][prop].append(val)
                    
    
    return calculateMajorityVote(grouped_votes)


directory = "results_run" 
results = organizeVotes(directory)
print(results)

{'gemini-2.5-pro_small_gru_compact_': {'name': 'São Paulo/Guarulhos International Airport', 'city': 'São Paulo'}, 'gemini-2.5-pro_big_gru_few3_': {'iata': 'GRU', 'name': 'São Paulo-Guarulhos International Airport', 'city': 'Guarulhos'}, 'gemini-2.5-pro_big_gru_few5_': {'iata': 'GRU', 'name': 'São Paulo/Guarulhos–Governador André Franco Montoro International Airport', 'city': 'Guarulhos'}, 'gemini-2.5-pro_big_gru_compact_': {'iata': 'GRU', 'name': 'São Paulo/Guarulhos–Governador André Franco Montoro International Airport', 'city': 'São Paulo'}, 'gemini-2.5-pro_bva_few5_': {'iata': 'BVA', 'name': 'Paris–Beauvais Airport', 'city': 'Beauvais'}, 'gemini-2.5-pro_small_gru_few5_': {'name': 'São Paulo/Guarulhos International Airport', 'city': 'São Paulo'}, 'gemini-2.5-pro_bva_few3_': {'iata': 'BVA', 'name': 'Paris–Beauvais Airport', 'city': 'Beauvais'}, 'gemini-2.5-pro_small_gru_few3_': {'name': 'São Paulo/Guarulhos International Airport', 'city': 'Guarulhos'}, 'gemini-2.5-pro_bva_compact_': {

## Generate ground truth

In [16]:
from neo4j import GraphDatabase
import pandas as pd

def getValuesForPropertiesFromNeo4j(entities, properties):
    
    NEO4J_URI = "neo4j://localhost:7687"
    NEO4J_USER = "neo4j"
    NEO4J_PASSWORD = "password"

    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    def fetch_nodes_and_props(tx, label):
        query = f"""
        MATCH (a:Airport)
        WHERE a.iata IN {entities}
        RETURN {properties}
        """
        # TODO change for icao later
        return list(tx.run(query))

    with driver.session(database="aiportcorrect") as session:
        nodes = session.execute_read(fetch_nodes_and_props, 'Airport')

    driver.close()

    return nodes


properties = 'a.iata, a.name, a.city'
entities = ['GRU','CDG']
results = getValuesForPropertiesFromNeo4j(entities, properties)

ground_truth = pd.DataFrame()

for index, record in enumerate(results):
    row = dict(record)
    ground_truth = pd.concat([ground_truth, pd.DataFrame([row])], ignore_index=True)

# Saving to CSV
ground_truth.to_csv('ground_truth.csv')


## Ranking accuracy

In [124]:
# da pra usar o que ja tem pronto das coisas de ML provavelmente.. 
# gerar por propriedade e depois no geral

import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_csv_per_model(pred_csv, truth_csv, tolerance=1e-4):
    pred = pd.read_csv(pred_csv)
    truth = pd.read_csv(truth_csv)

    pred["model"] = pred["model"].astype(str)

    pred_no_model = pred.drop(columns=["model"])
    truth = truth.drop(columns=["id"])

    models = pred["model"].unique()
    cols = truth.columns

    final_results = {}

    for model_name in models:
        model_mask = pred["model"] == model_name
        
        pred_model = pred_no_model[model_mask].reset_index(drop=True)
        truth_model = truth.reset_index(drop=True)

      
        if len(pred_model) != len(truth_model):
            raise ValueError(
                f"Model '{model_name}' has {len(pred_model)} predictions "
                f"but ground truth has {len(truth_model)} rows."
            )

        results = {}

        for col in cols:
            pred_values = pred_model[col].to_numpy()
            truth_values = truth_model[col].to_numpy()

            if truth[col].dtype == float or truth[col].dtype == int:
                correct = (abs(pred_values - truth_values) <= tolerance).astype(int)
            else:
                pred_norm = pd.Series(pred_values).astype(str).str.strip().str.lower().to_numpy()
                truth_norm = pd.Series(truth_values).astype(str).str.strip().str.lower().to_numpy()

                correct = (pred_norm == truth_norm).astype(int)

            y_true = [1] * len(correct)
            y_pred = correct

            results[col] = {
                "accuracy": accuracy_score(y_true, y_pred),
                "precision": precision_score(y_true, y_pred),
                "recall": recall_score(y_true, y_pred),
                "f1": f1_score(y_true, y_pred),
                "num_correct": correct.sum(),
                "num_total": len(correct)
            }

        final_results[model_name] = results

    return final_results

metrics = evaluate_csv_per_model("result_test1.csv", "ground_truth.csv")

print("\n=== Metrics per model ===")
for model, m in metrics.items():
    print(f"\nModel: {model}")
    print(m)

# for col, m in metrics.items():
#     print(f"\nColuna: {col}")
#     for metric_name, value in m.items():
#         print(f"  {metric_name}: {value:.4f}")


ValueError: Model 'gemini1' has 2 predictions but ground truth has 1 rows.

In [132]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score


def evaluate_with_sklearn(result_csv, ground_truth_csv, key_column="iata"):

    gt = pd.read_csv(ground_truth_csv)
    results = pd.read_csv(result_csv)

    gt = gt.rename(columns={c: c.split(".")[-1] for c in gt.columns})

    common_columns = [
        c for c in results.columns
        if c in gt.columns and c not in ["model", key_column]
    ]

    def norm(x):
        if pd.isna(x):
            return ""
        return str(x).strip().lower()

    gt[key_column] = gt[key_column].apply(norm)
    results[key_column] = results[key_column].apply(norm)

    metrics = []

    for _, row in results.iterrows():
        model_name = row["model"]
        pred_key = row[key_column]

        gt_row = gt[gt[key_column] == pred_key]

        if gt_row.empty:
            y_true = [""] * len(common_columns)
        else:
            gt_row = gt_row.iloc[0]
            y_true = [norm(gt_row[col]) for col in common_columns]

        y_pred = [norm(row[col]) for col in common_columns]

        col_acc = {}
        col_f1 = {}
        for i, col in enumerate(common_columns):
            col_acc[f"accuracy_{col}"] = accuracy_score([y_true[i]], [y_pred[i]])
            col_f1[f"f1_{col}"] = f1_score([y_true[i]], [y_pred[i]], average="binary", pos_label=y_true[i])

        overall_accuracy = accuracy_score(y_true, y_pred)
        overall_f1 = f1_score(y_true, y_pred, average="macro")

        metrics.append({
            "model": model_name,
            **col_acc,
            **col_f1,
            "overall_accuracy": overall_accuracy,
            "overall_f1_macro": overall_f1
        })

    return pd.DataFrame(metrics)

df_metrics = evaluate_with_sklearn("airports_dynamic.csv", "ground_truth.csv")
print(df_metrics)


                               model  accuracy_city  accuracy_name  f1_city  \
0  gemini-2.5-pro_small_gru_compact_            0.0            0.0      0.0   
1       gemini-2.5-pro_big_gru_few3_            0.0            0.0      0.0   
2       gemini-2.5-pro_big_gru_few5_            0.0            0.0      0.0   
3    gemini-2.5-pro_big_gru_compact_            0.0            0.0      0.0   
4           gemini-2.5-pro_bva_few5_            0.0            0.0      0.0   
5     gemini-2.5-pro_small_gru_few5_            0.0            0.0      0.0   
6           gemini-2.5-pro_bva_few3_            0.0            0.0      0.0   
7     gemini-2.5-pro_small_gru_few3_            0.0            0.0      0.0   
8        gemini-2.5-pro_bva_compact_            0.0            0.0      0.0   

   f1_name  overall_accuracy  overall_f1_macro  
0      0.0               0.0               0.0  
1      0.0               0.0               0.0  
2      0.0               0.0               0.0  
3      0.0   